In [2]:
import os

from azureml.core import  Environment
from azureml.data.data_reference import DataReference
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azureml.core import Workspace, Dataset

Azure ML SDK Version:  1.51.0


In [4]:
try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
    print(credential)
except Exception as ex:
    credential = InteractiveBrowserCredential()

In [5]:
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


In [6]:
ws = Workspace.from_config()

In [ ]:
datastore = ws.get_default_datastore()

### Create enviroment

In [ ]:
env = Environment.from_pip_requirements(name='local-env', file_path='requirements.txt')

# Register the environment
env.register(workspace=ws)

### Load data from datastore and register it as dataset

In [31]:
data_reference = DataReference(datastore, path_on_datastore='UI/2024-05-25_135438_UTC/archive.zip')
dataset = Dataset.File.from_files(path=data_reference)
dataset.register(ws, name='image_cards_dataset', create_new_version=True)

### Load dataset

In [36]:
dataset = Dataset.get_by_name(ws, 'image_cards_dataset')
local_path = dataset.download(target_path='./data', overwrite=True)

In [9]:
extract_path = os.path.join('data', 'prepared_data')

### Unzip data

In [39]:
import zipfile


# Unzip the file
with zipfile.ZipFile(local_path[0], 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f'Files extracted to: {extract_path}')

Files extracted to: data/prepared_data


### Creating data asset from unzipped images

In [21]:
from azure.ai.ml.entities import Data

my_data = Data(
    path='./dataaa/prepared_data',  
    type=AssetTypes.URI_FOLDER,     
    description="Prepared cards data",
    name="prepared_cards_images_dataset",
    version="1",
)
# Create the data asset in the workspace
ml_client.data.create_or_update(my_data)

In [ ]:
data_asset_name = "prepared_cards_images_dataset"
data_asset_version = "1"

data_asset = ml_client.data.get(name=data_asset_name, version=data_asset_version)